# [1] Scientific computation

There are several packages that provide multidimensional data manipulation, optimization, regression, interpolation and visualization, among other possibilities.

## Some floating point arithmetic insights
Python uses (hardware) 754 double precision for floats. This means that some floats can be only represented approximately.

In [2]:
format(0.1, '.20f')

'0.10000000000000000555'

For "infinite" precision float arithmetic you can use [decimal](https://docs.python.org/3/library/decimal.html#module-decimal).

In [4]:
from decimal import Decimal
format(Decimal(1)/Decimal(10), '.20f')

'0.10000000000000000000'

Lets compute the $\pi$ number using the [Bailey–Borwein–Plouffe formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula):

$$
\pi = \sum_{k = 0}^{\infty}\left[ \frac{1}{16^k} \left( \frac{4}{8k + 1} - \frac{2}{8k + 4} - \frac{1}{8k + 5} - \frac{1}{8k + 6} \right) \right]
$$

In [3]:
# https://stackoverflow.com/questions/28284996/python-pi-calculation
from decimal import Decimal, getcontext
getcontext().prec=100
my_pi= sum(1/Decimal(16)**k * 
          (Decimal(4)/(8*k+1) - 
           Decimal(2)/(8*k+4) -
           Decimal(1)/(8*k+5) -
           Decimal(1)/(8*k+6)) for k in range(100))
'{:.100f}'.format(my_pi)

'3.1415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170700'

You can visit [100,000 Digits of Pi](http://www.geom.uiuc.edu/~huberty/math5337/groupe/digits.html) to test if this code works or not.

## 1. SciPy.org's [Numpy](http://www.numpy.org/)

Numpy provides a high-performance multidimensional array object.

### 1.1. Installation

```
pip install numpy
```

### 1.2. Why numpy?

Better running times:

In [4]:
import numpy as np

l = list(range(0,100000)); print(type(l), l[:10])
a = np.arange(0, 100000); print(type(a), a[:10])

(<type 'list'>, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(<type 'numpy.ndarray'>, array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))


In [47]:
%timeit sum(l)

1000 loops, best of 3: 1.32 ms per loop


In [48]:
%timeit np.sum(a)

10000 loops, best of 3: 78.5 µs per loop


In [1]:
!cat sum_array.c
!gcc -O3 sum_array.c -o sum_array
%timeit !./sum_array

#include <stdio.h>
#include <time.h>

#define N 100000

int main() {
  int a[N];
  int i;
  clock_t start, end;
  double cpu_time;
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  start = clock();
  long int sum = 0;
  for(i=0; i<N; i++) {
    sum += *a+i;
  }
  end = clock();
  printf("%ld ", sum);
  cpu_time = ((double) (end - start)) / CLOCKS_PER_SEC;
  cpu_time *= 1000000;
  printf("%f usegs\n", cpu_time);
}
4999950000 4.000000 usegs
4999950000 4.000000 usegs
4999950000 3.000000 usegs
4999950000 4.000000 usegs
1 loop, best of 3: 119 ms per loop


Looking for something:

In [4]:
np.lookfor('invert')

Search results for 'invert'
---------------------------
numpy.bitwise_not
    Compute bit-wise inversion, or bit-wise NOT, element-wise.
numpy.matrix.getI
    Returns the (multiplicative) inverse of invertible `self`.
numpy.in1d
    Test whether each element of a 1-D array is also present in a second array.
numpy.transpose
    Permute the dimensions of an array.
numpy.linalg.inv
    Compute the (multiplicative) inverse of a matrix.
numpy.linalg.pinv
    Compute the (Moore-Penrose) pseudo-inverse of a matrix.
numpy.linalg.tensorinv
    Compute the 'inverse' of an N-dimensional array.


You can also use the tabulator to extend some command of use a wildcard in Ipython:

In [6]:
np.*?

### 1.3. Creating arrays
A numpy array is a grid of values, all of the same type, indexed by a tuple of nonnegative integers.

### 1.3.1. 1D

Creation:

In [5]:
a = np.array([1, 2, 3])  # Create a rank 1 array
print(type(a))

<class 'numpy.ndarray'>


In [6]:
print(a.ndim)

1


In [7]:
print(a)

[1 2 3]


In [8]:
print(a.shape)

(3,)


In [9]:
print(len(a))

3


In [5]:
np.linspace(1., 4., 6)

array([ 1. ,  1.6,  2.2,  2.8,  3.4,  4. ])

In [6]:
!cat data.txt
np.genfromtxt('data.txt')

1	200
2	150
3	250


array([[   1.,  200.],
       [   2.,  150.],
       [   3.,  250.]])

### Indexing:

In [10]:
print(a[0], a[1])

1 2


In [ ]:
a[0] = 0
print(a)

Arrays can be creted from different types of contaniers (which store complex numbers in this case):

In [7]:
x = np.array([[1,1.0],(1+1j,.3)])
x

array([[ 1.0+0.j,  1.0+0.j],
       [ 1.0+1.j,  0.3+0.j]])

### 1.3.2. 2D

With 2 arrays:

In [52]:
b = np.array([[1,2,3],[4,5,6]])   # Create a rank 2 array
print(b)
print(b.shape)
print(b[1, 1])

[[1 2 3]
 [4 5 6]]
(2, 3)
5


With zeroes:

In [53]:
a = np.zeros((5,5)) # The default dtype is float64
print(a)

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


With ones:

In [54]:
a = np.ones((5,5))
print(a)

[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]


With an arbitrary scalar:

In [55]:
a = np.full((5,5), 2)
print(a)

[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]]


The identity matrix:

In [56]:
a = np.eye(5)
print(a)

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


With ramdom data:

In [57]:
a = np.random.random((5,5))
print(a)

[[ 0.58571528  0.25685412  0.01261491  0.00878228  0.02077394]
 [ 0.74879231  0.18543502  0.13214     0.73373718  0.35608532]
 [ 0.84217412  0.75338056  0.64339627  0.56303606  0.35189481]
 [ 0.75285607  0.08173016  0.99074363  0.23117919  0.83229632]
 [ 0.48143311  0.0580023   0.41653061  0.77619903  0.37963861]]


In [58]:
a = np.random.random((5,5))
print(a)

[[ 0.75978992  0.8906945   0.5182052   0.63130611  0.72239936]
 [ 0.53203257  0.51611032  0.57183896  0.51173231  0.982197  ]
 [ 0.01866067  0.28525326  0.4658827   0.87065438  0.59222394]
 [ 0.2878274   0.59260858  0.5998953   0.11517093  0.05926045]
 [ 0.40350625  0.56051949  0.02390184  0.12645223  0.33227536]]


Filled with zeroes and with a previously defined shape:

In [59]:
b = np.empty_like(a)
print(b)

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


With a 1D list comprehension:

In [60]:
a = np.array([i for i in range(5)])
print(a, a[1], a.shape)

[0 1 2 3 4] 1 (5,)


With a 2D list comprehension:

In [61]:
b = np.array([[j for j in range(10)] for i in range(1)])
print(b, b.shape)

[[0 1 2 3 4 5 6 7 8 9]] (1, 10)


A different 2D list comprehension:

In [62]:
a = np.array([[i*5+j for j in range(5)] for i in range(5)])
print(a)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


Getting elements of a matrix:

In [63]:
print(a[0,3], a[1,2], a[2,1])

3 7 11


Getting elements of a matrix using "integer array indexing":

In [64]:
print(a)
print(a[[0, 1, 2], [3, 2, 1]])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[ 3  7 11]


The same integer array indexing using comprehension lists:

In [65]:
print(a[np.array([i for i in range(3)]), np.array([i for i in range(3,0,-1)])])

[ 3  7 11]


The same using the `np.arange()` function:

In [66]:
print(np.arange(3))
print(np.arange(3,0,-1))
print(a[np.arange(3), np.arange(3,0,-1)])

[0 1 2]
[3 2 1]
[ 3  7 11]


### 1.4. Slicing

In [67]:
print(a,a[0:],a[1:])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]] [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]] [[ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


Getting a top-left submatrix:

In [68]:
print(a[:2,:2])

[[0 1]
 [5 6]]


Getting a bottom-right submatrix:

In [69]:
print(a[2:,2:])

[[12 13 14]
 [17 18 19]
 [22 23 24]]


Getting a column of a matrix:

In [70]:
print("column 2 =", a[:,2])

column 2 = [ 2  7 12 17 22]


Sampling:

In [71]:
print(a[2::2,::2])

[[10 12 14]
 [20 22 24]]


 ### 1.5. Boolean array indexing

Finding the elements bigger than ...

In [72]:
bool_idx = (a>12)
print(bool_idx)

[[False False False False False]
 [False False False False False]
 [False False False  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


In [73]:
print(a[bool_idx])

[13 14 15 16 17 18 19 20 21 22 23 24]


### 1.6. Elementwise (vectorial-vectorial and vectorial-scalar) math

In [74]:
a = np.zeros((5,5), np.int32)
print(a)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [75]:
a[1:4,1:4] = 1
print(a)

[[0 0 0 0 0]
 [0 1 1 1 0]
 [0 1 1 1 0]
 [0 1 1 1 0]
 [0 0 0 0 0]]


Vectorial-scalar addition:

In [76]:
a[1:4, 1:4] += 1
print(a)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


In [77]:
b = np.ones((5,5), np.int32)
print(b)

[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]


Vectorial addition:

In [78]:
c = a + b
print(c)

[[1 1 1 1 1]
 [1 3 3 3 1]
 [1 3 3 3 1]
 [1 3 3 3 1]
 [1 1 1 1 1]]


Vectorial substraction:

In [79]:
d = c - b
print(d)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


Vectorial multiplication (not matrix multiplication!):

In [80]:
c = c * d
print(c)

[[0 0 0 0 0]
 [0 6 6 6 0]
 [0 6 6 6 0]
 [0 6 6 6 0]
 [0 0 0 0 0]]


Floating-point vectorial division:

In [81]:
c = c / b
print(c)

[[ 0.  0.  0.  0.  0.]
 [ 0.  6.  6.  6.  0.]
 [ 0.  6.  6.  6.  0.]
 [ 0.  6.  6.  6.  0.]
 [ 0.  0.  0.  0.  0.]]


Fixed-point (integer) vectorial division:

In [82]:
c = d // b
print(c)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


### Broadcasting
In vectorized operations, NumPy "extends" scalars and arrays with one of its dimensions equal to 1 to the size of the other(s) array(s).

In [8]:
a = np.ones((5,3))
a

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [14]:
b = np.arange(1) + 1
b

array([1])

In [15]:
a+b # 'a' is 5x3 and 'b' is 1x1

array([[ 2.,  2.,  2.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  2.]])

In [16]:
b = np.arange(3)
b

array([0, 1, 2])

In [17]:
a+b # 'a' is 5x3 and 'b' is '1x3'

array([[ 1.,  2.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  2.,  3.]])

In [50]:
b = np.arange(5)
b

array([0, 1, 2, 3, 4])

In [56]:
b = b.reshape((5,1)) # (Rows, Columns)
b

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [57]:
a+b

array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.],
       [ 3.,  3.,  3.],
       [ 4.,  4.,  4.],
       [ 5.,  5.,  5.]])

If the arrays have different shapes and s can not be "broadcasted", `ValueError: frames are not aligned` is thrown.

In [60]:
b = np.arange(4)[:, None]
b

array([[0],
       [1],
       [2],
       [3]])

In [61]:
a.shape

(5, 3)

In [62]:
b.shape

(4, 1)

In [63]:
a+b

ValueError: operands could not be broadcast together with shapes (5,3) (4,1) 

### 1.7. Matricial math
Provides basic matrix computation.

A chessboard matrix:

In [ ]:
a = np.array([[(i+j)%2 for j in range(10)] for i in range(10)])
print(a, a.shape)

... and a 1-column matrix:

In [ ]:
b = np.array([[1] for i in range(10)])
print(b, b.shape)

Product matrix-vector:

In [ ]:
c = np.dot(a,b)
print(c)

Sum of all elements of a matrix:

In [ ]:
print(np.sum(c))

In [ ]:
print(np.sum(a))

Compute the maximum of a matrix:

In [ ]:
print(np.max(c))

Matrix transpose:

In [ ]:
print(c.T, c.T.shape, c.shape)

### How fast is array math?

In [ ]:
a = np.array([[(i+j) for j in range(10)] for i in range(10)])
print(a, a.shape)

In [ ]:
a[:1]

In [ ]:
a[:1].shape

In [ ]:
a[:1][0]

In [ ]:
a[:1][0].shape

In [ ]:
b = a[:1][0]
print(b, b.shape)

Add `b[]` to all the rows of `a[][]` using scalar arithmetic:

In [ ]:
c = np.empty_like(a)
def add():
    for i in range(a.shape[1]):
        for j in range(a.shape[0]):
            c[i, j] = a[i, j] + b[j]
%timeit add()
print(c)

Add b[] to all the rows of a[][] using vectorial arithmetic:

In [ ]:
c = np.empty_like(a)
def add():
    for i in range(a.shape[1]):
        c[i, :] = a[i, :] + b
%timeit add()
print(c)

Add b[] to all the rows of a[][] using fully scalar arithmetic:

In [ ]:
%timeit c = a + b # <- broadcasting is faster
print(c)

### Structured arrays

In [65]:
x = np.array([(1,2.,'Hello'), (3,4.,"World")],
             dtype=[('first', 'i4'),('second', 'f4'), ('third', 'S10')])
x

array([(1,  2., 'Hello'), (3,  4., 'World')],
      dtype=[('first', '<i4'), ('second', '<f4'), ('third', 'S10')])

In [67]:
x['first']

array([1, 3], dtype=int32)

In [68]:
x['second']

array([ 2.,  4.], dtype=float32)

In [69]:
x['third']

array(['Hello', 'World'],
      dtype='|S10')

## 2. [Matplotlib](http://matplotlib.org)
A Python 2D plotting library.

### 2.1. Installation

```
pip install matplotlib
```

### 2.2. Matplotlib in-line of Ipython (Jupyter) notebook

In [ ]:
%matplotlib inline

### 2.3. Importing it

In [ ]:
import matplotlib.pyplot as plt

### 2.4. Drawing data structures (matrices):

In [ ]:
chess_board = np.zeros([8, 8], dtype=int)
chess_board[0::2, 1::2] = 1
chess_board[1::2, 0::2] = 1
plt.matshow(chess_board, cmap=plt.cm.gray)

### 2.5. Drawing 2D curves

In [ ]:
resolution = 100
x = np.arange(0, 3*np.pi, np.pi/resolution)
si = np.sin(x)
co = np.cos(x)
plt.plot(x, si, c = 'r')
plt.plot(x, co, c = 'g')
plt.legend(['$\sin(x)$', '$\cos(x)$'])
plt.xlabel('radians')
plt.title('sine($x$) vs. cosine($x$)')
plt.xticks(x*resolution, ['0', '$\pi$', '$2\pi$'], rotation='horizontal')
plt.xlim(0,3*np.pi)
plt.show()

### 2.6. Drawing 3D curves

In [ ]:
x = np.array([[(x+y)/25 for x in range(256)] for y in range(256)])
si = np.sin(x)
plt.imshow(si, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
# https://github.com/AeroPython/Taller-Aeropython-PyConEs16
def funcion(x,y):
    return np.cos(x) + np.sin(y)

x_1d = np.linspace(0, 5, 100)
y_1d = np.linspace(-2, 4, 100)
X, Y = np.meshgrid(x_1d, y_1d)
Z = funcion(X,Y)
plt.contourf(X, Y, Z, np.linspace(-2, 2, 100),cmap=plt.cm.Spectral)
plt.colorbar()
cs = plt.contour(X, Y, Z, np.linspace(-2, 2, 9), colors='k')
plt.clabel(cs)

## 3. [SciPy](https://docs.scipy.org/doc/scipy/reference/)
[SciPy](http://cs231n.github.io/python-numpy-tutorial/#numpy-array-indexing) provides a large number of functions that operate on numpy arrays and are useful for different types of scientific and engineering applications such as:
1. [Custering](https://docs.scipy.org/doc/scipy/reference/cluster.html).
2. [Discrete Fourier Analysis](https://docs.scipy.org/doc/scipy/reference/fftpack.html).
3. [Interpolation](https://docs.scipy.org/doc/scipy/reference/interpolate.html).
4. [Linear algebra](https://docs.scipy.org/doc/scipy/reference/linalg.html).
5. [Signal](https://docs.scipy.org/doc/scipy/reference/signal.html) and [Image processing](https://docs.scipy.org/doc/scipy/reference/ndimage.html).
6. [Optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html).
7. [Sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html) and [sparse linear algebra](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html).



### 3.1. Installation

```
pip install scipy
```

### 3.1.1. Optimization example

In [ ]:
# http://www.scipy-lectures.org/advanced/mathematical_optimization/
from scipy import optimize

def f(x):
    return -np.exp(-(x - .7)**2)

In [ ]:
sol = optimize.brent(f)
print('min =', sol, '\nx =', f(sol))

In [ ]:
x = np.arange(-10, 10, 0.1)
plt.plot(x, f(x))
plt.plot([sol],[f(sol)], 'ro')
plt.show()

## 4. [Pandas](http://pandas.pydata.org/)
High-performance data structures and data analysis tools for the Python programming language (similar to [R](https://en.wikipedia.org/wiki/R_(programming_language)). Some tools are:
1. [Statistical functions (covariance, correlation)](http://pandas.pydata.org/pandas-docs/stable/computation.html#statistical-functions).
2. [Window functions](http://pandas.pydata.org/pandas-docs/stable/computation.html#window-functions).
3. [Time series](http://pandas.pydata.org/pandas-docs/stable/timeseries.html).
4. [Analysis of sparse data](http://pandas.pydata.org/pandas-docs/stable/sparse.html).

### 4.1. Installation

```
pip3 install pandas
```

### 4.2. Example

Create a table with data:

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'int_col' : [1, 2, 6, 8, -1],
                    'float_col' : [0.1, 0.2, 0.2, 10.1, None],
                    'str_col' : ['a', 'b', None, 'c', 'a']})
print(df)
df

Arithmetic average of a column:

In [ ]:
df2 = df.copy()
mean = df2['float_col'].mean()
mean

Replace undefined elements:

In [ ]:
df3 = df['float_col'].fillna(mean)
df3

Create a table by means of columns:

In [ ]:
df4 = pd.concat([df3, df['int_col'], df['str_col']], axis=1)
df4

## 5. [SymPy](http://www.sympy.org/en/index.html)
A Python library for symbolic mathematics. Among others things, it provides:
1. [Symbolic simplification](http://docs.sympy.org/latest/tutorial/simplification.html).
2. [Calculus (derivatives, integrals, limits, and series expansions)](http://docs.sympy.org/latest/tutorial/calculus.html).
3. [Algebraic solver](http://docs.sympy.org/latest/tutorial/solvers.html).
4. [Matrix operations](http://docs.sympy.org/latest/tutorial/matrices.html).
5. [Combinatorics](http://docs.sympy.org/latest/modules/combinatorics/index.html)
6. [Cryptography](http://docs.sympy.org/latest/modules/crypto.html).

### 5.1. Install
```
pip install sympy
```

### 5.2. Example

In [ ]:
from sympy import init_session
init_session(use_latex='matplotlib')

In [ ]:
# https://github.com/AeroPython/Taller-Aeropython-PyConEs16
expr = cos(x)**2 + sin(x)**2
expr

In [ ]:
simplify(expr)

In [ ]:
expr.subs(x, y**2)

In [ ]:
expr = (x + y) ** 2
expr

In [ ]:
expr = expr.expand()
expr

In [ ]:
expr = expr.factor()
expr

In [ ]:
expr = expr.integrate(x)
expr

In [ ]:
expr = expr.diff(x)
expr